In [ ]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
import os 
import pandas as pd
import dateutil
import sklearn.preprocessing as preprocess
import numpy as np
import pdb
pd.options.display.max_columns  = 1000

In [ ]:
data_dir = os.path.join(os.getcwd(),"data")
if not os.path.exists(data_dir):
    os.mkdir(data_dir)
nba_teams = teams.get_teams()
for team in nba_teams:
    abbrev = team['abbreviation']
    fn = os.path.join(data_dir, abbrev+".csv")
    if not os.path.exists(fn):
        team_id = team['id']
        gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id)
        df = gamefinder.get_data_frames()[0]
        df.to_csv(os.path.join(data_dir, abbrev + ".csv"), index=False)

Clean it 

In [ ]:
#combine into one df 
dfs = [None]*len(nba_teams)
for idx, team in enumerate(nba_teams):
    abbrev = team['abbreviation']
    dfs[idx] = pd.read_csv(os.path.join(data_dir, abbrev+".csv"))
df = pd.concat(dfs,ignore_index=True)
df = df.drop(['PLUS_MINUS', 'FG3_PCT','FG_PCT','FT_PCT','WL'],axis=1)#there's missing vas or NANS (divide by 0 for pct)
display(df.head())

There are still some rows with missing values, but considering there are 40,000 games 10 is very good. I'm going
to set these to 0 

In [ ]:
df.shape[0]/2

In [ ]:
naMask = np.any(df.isna().values, axis=1)
df.loc[naMask]

In [ ]:
df.fillna(0, inplace=True)
df.loc[naMask]

The GAME_IDs are not perfect. As seen below, there can be 1-4 instances 
with the same GAME_ID; ideally there should be 2 - one for each team that plays in that game. We can see that some games with overtimes are counted twice (which make

In [ ]:
UID, count = np.unique(df['GAME_ID'], return_counts=True)
notTWO = count != 2
badUIDs = UID[notTWO]
display(count[notTWO])
isBadMask = np.isin(df['GAME_ID'], UID[notTWO])
display(df[isBadMask].sort_values(by=['GAME_ID']))

In [ ]:
sum(isBadMask)/df.shape[0] #only .3% of the data is bad so going to throw it out

In [ ]:
drop1 = df[np.logical_not(isBadMask)].copy()

In [ ]:
#ok we also need to verify that of gameIDs with two instances those instances belong to separate teams
teams = pd.unique(drop1['TEAM_ABBREVIATION'])
badIDs = []
for team in teams:
    team_df = drop1[drop1['TEAM_ABBREVIATION'] == team]
    ids, counts = np.unique(team_df['GAME_ID'].values, return_counts=True)
    badIDs += list(ids[counts > 1])
    if np.any(counts > 1):
        print(team)
    
print(badIDs) 
    

In [ ]:
#so only one bad ID
drop2 = drop1[np.logical_not(np.isin(drop1['GAME_ID'], badIDs))].copy()

In [ ]:
print(drop2.size)
display(drop2.head())

In [ ]:
#ok at this point we know that there are two teams per GAME_ID and those teams are different. There are also
#no missing data. 
#we'll save this cleaned data

drop2.to_csv(os.path.join(data_dir,"cleanNBA.csv"), index=False)

In [ ]:
drop2.shape[0]/2

In [ ]:
pd.options.display.max_columns = 15
drop2[drop2['GAME_ID']==21900969]

In [ ]:
teams = np.unique(drop2['TEAM_NAME'])

In [ ]:
teams